In [1]:
#!pip install selenium
#!pip install webdriver_manager
#!pip install openpyxl

In [2]:
username_linkedin = ""
password_linkedin = ""
keyword = "Machine Learning Engineer"
location = "France"
post_ages = 1 # job posted last 1 days
time_sleeps = 1 # wait 1 seconds before each action

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from typing import Union
import urllib.parse

In [4]:
def get_job_search_url(search_period: Union[int, None] = None):
    encoded_url_key_word = urllib.parse.quote(keyword)
    encoded_url_location = urllib.parse.quote(location)
    if type(search_period) is int:
        return f'https://www.linkedin.com/jobs/search/?f_TPR=r{search_period*86400}&keywords={encoded_url_key_word}&location={encoded_url_location}'
    else:
        return f'https://www.linkedin.com/jobs/search/?keywords={encoded_url_key_word}&location={encoded_url_location}'

In [5]:
s=Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(service=s,options=options)
driver.maximize_window()
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

time.sleep(time_sleeps)
username = WebDriverWait(driver, time_sleeps).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='session_key']")))
password = WebDriverWait(driver, time_sleeps).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='session_password']")))

username.clear()
username.send_keys(username_linkedin)
password.clear()
password.send_keys(password_linkedin)

time.sleep(time_sleeps)
button = WebDriverWait(driver, time_sleeps).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

print("You are logged in!")


You are logged in!


In [6]:
url = get_job_search_url(post_ages)
print(f'Search url: {url}')

Search url: https://www.linkedin.com/jobs/search/?f_TPR=r86400&keywords=Machine%20Learning%20Engineer&location=France


In [7]:
driver.get(url)
time.sleep(time_sleeps)

In [8]:
total_results = WebDriverWait(driver, time_sleeps).until(EC.visibility_of_all_elements_located((By.XPATH, ".//small")))
if len(total_results) == 1:
    print(total_results[0].text)

51 results


In [9]:
number_of_pages = int(total_results[0].text.split()[0].replace(",","")) // 25
print(f'{int(number_of_pages)} pages')

2 pages


In [10]:
job_ids = []
job_titles = []
company_names = []
job_ages = []
job_descriptions = []

In [11]:
def scrap_data():
    search_results = WebDriverWait(driver, time_sleeps).until(EC.visibility_of_all_elements_located((By.XPATH, './/ul[@itemtype="http://schema.org/ItemList"]')))
    jobs = search_results[0].find_elements(By.XPATH, ".//li[@data-occludable-job-id]")

    for job in jobs:
        job_ids.append(job.get_attribute('data-occludable-job-id'))

        job.click()

        job_description = WebDriverWait(driver, time_sleeps).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[contains(@class, 'jobs-description--reformatted')]")))
        job_descriptions.append(job_description[0].text)

        job_title = WebDriverWait(driver, time_sleeps).until(EC.visibility_of_all_elements_located((By.XPATH, "//h2[contains(@class, 'jobs-unified-top-card__job-title')]")))
        job_titles.append(job_title[0].text)

        company_name = WebDriverWait(driver, time_sleeps).until(EC.visibility_of_all_elements_located((By.XPATH, "//span[contains(@class, 'jobs-unified-top-card__company-name')]")))
        company_names.append(company_name[0].text)

        job_age = WebDriverWait(driver, time_sleeps).until(EC.visibility_of_all_elements_located((By.XPATH, "//span[contains(@class, 'jobs-unified-top-card__posted-date')]")))
        job_ages.append(job_age[0].text)

        time.sleep(time_sleeps)


In [12]:
scrap_data()
for i in range(number_of_pages):
    new_url = f'{url}&start={(i+1)*25}'
    print(f'Search url: {url}')
    driver.get(new_url)
    time.sleep(time_sleeps)
    scrap_data()

Search url: https://www.linkedin.com/jobs/search/?f_TPR=r86400&keywords=Machine%20Learning%20Engineer&location=France
Search url: https://www.linkedin.com/jobs/search/?f_TPR=r86400&keywords=Machine%20Learning%20Engineer&location=France


In [13]:
import pandas as pd
data = {
    'job_id': job_ids,
    'job_title': job_titles,
    'company_name': company_names,
    'job_age': job_ages,
    'job_description': job_descriptions
}
df = pd.DataFrame(data, columns=['job_id', 'job_title', 'company_name', 'job_age', 'job_description'])
df

,job_id,job_title,company_name,job_age,job_description
0,3183379236,Machine Learning Engineer I,American Express Global Business Travel,1 hour ago,Would you like to be in the forefront of innov...
1,3185831024,Senior Machine Learning Engineer - Moteur de r...,Lity,20 hours ago,C'est le moteur de recherche Franco-Americain ...
2,3185827333,Software Engineer at medGo,join.com,20 hours ago,medGo’s vision: change the daily life of 10 mi...
3,3183379237,Data Engineer II,American Express Global Business Travel,1 hour ago,As a member of Egencia’s Data Engineering grou...
4,2900026537,JOIN OUR RETURNSHIP PROGRAM - Software Develop...,Criteo,14 hours ago,"Who We Are\n\nAt Criteo, our culture is as uni..."
5,3084411317,Data Scientist,Solvay,14 hours ago,Solvay is a science company whose technologies...
6,3044688621,Software Development Engineer,Criteo,14 hours ago,"Who We Are\n\nAt Criteo, our culture is as uni..."
7,3184156649,Backend Services . Software Engineer,Jobs via eFinancialCareers,8 hours ago,
8,2971505135,C++ Software Engineer - Engine Programmer H/F/NB,Ubisoft Paris Studio,11 hours ago,
9,3187399049,Senior Full-Stack Software Engineer (m/f/d),Global Savings Group,3 hours ago,Global Savings Group is Europe's largest and m...


In [14]:
#saving the data
df.to_excel(f'Linkedin_Job_{keyword}.xlsx', index = False)

In [15]:
driver.close()
print("Your search is done!")

Your search is done!
